In [289]:
import pandas as pd
from datetime import datetime
from functools import reduce

loans_df = pd.read_csv('../data_generators/output/loans.csv')
spend_df = pd.read_csv('../data_generators/output/monthly_spend.csv')

In [290]:
loans_df.columns = [column.strip() for column in loans_df.columns]
loans_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 296 entries, 0 to 295
Data columns (total 10 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   customer_id            296 non-null    object
 1   first_name             296 non-null    object
 2   last_name              296 non-null    object
 3   loan_id                296 non-null    object
 4   loan_category          296 non-null    object
 5   nr_of_months           296 non-null    int64 
 6   due_amount_in_usd      296 non-null    int64 
 7   due_date               296 non-null    object
 8   payment_date           296 non-null    object
 9   payment_amount_in_usd  296 non-null    int64 
dtypes: int64(3), object(7)
memory usage: 23.2+ KB


In [291]:
spend_df.columns = [column.strip() for column in spend_df.columns]
spend_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1200 entries, 0 to 1199
Data columns (total 8 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   customer_id             1200 non-null   object
 1   first_name              1200 non-null   object
 2   last_name               1200 non-null   object
 3   spend_rate              1200 non-null   object
 4   available_money_rate    1200 non-null   object
 5   spend_money_in_usd      1200 non-null   int64 
 6   available_money_in_usd  1200 non-null   int64 
 7   month                   1200 non-null   object
dtypes: int64(2), object(6)
memory usage: 75.1+ KB


In [292]:
loans_df.head(2)

,customer_id,first_name,last_name,loan_id,loan_category,nr_of_months,due_amount_in_usd,due_date,payment_date,payment_amount_in_usd
0,169107568X,Christopher,Turner,08196e933365451cb5f51011d20cc903,Auto,30,258,26-04-2015,2-04-2015,251
1,169107568X,Christopher,Turner,08196e933365451cb5f51011d20cc903,Auto,30,258,26-05-2015,12-05-2015,282


In [293]:
grouped_by_customer = loans_df.groupby(['customer_id'])

In [294]:
loan_debtor_due_amount = grouped_by_customer.apply(lambda x: x['due_amount_in_usd'].sum())
loan_debtor_due_amount

customer_id
0254558577    8225
0448103850    2016
0578871866    7334
0627892167    5400
1259477169    5580
169107568X    7740
1737533871    8084
1767857853    3500
1777845416    5418
1978016417    5597
dtype: int64

In [295]:
loan_debtor_paid_amount = grouped_by_customer.apply(lambda x: x['payment_amount_in_usd'].sum())
loan_debtor_paid_amount

customer_id
0254558577    7994
0448103850    2031
0578871866    7236
0627892167    5351
1259477169    5435
169107568X    7801
1737533871    7734
1767857853    3480
1777845416    4935
1978016417    5677
dtype: int64

In [296]:
loan_debtors_balance = loan_debtor_due_amount - loan_debtor_paid_amount
loan_debtors_balance_df = loan_debtors_balance.to_frame()
loan_debtors_balance_df.columns = ['debtors']
loan_debtors_balance_df['debtors'] = loan_debtors_balance_df['debtors'].map(lambda x: 0 if x >= 0 else 3)  # plus 3 points if positive int
loan_debtors_balance_df

,debtors
customer_id,
0254558577,0
0448103850,3
0578871866,0
0627892167,0
1259477169,0
169107568X,3
1737533871,0
1767857853,0
1777845416,0


In [297]:
def count_points_for_skippers(value):
    if value > 2:
        return 1
    return 0

In [298]:
loans_skippers = grouped_by_customer.apply(lambda x: x.loc[x['payment_amount_in_usd'] == 0])
loans_skippers_count = loans_skippers['customer_id'].value_counts()
loans_skippers_df = loans_skippers_count.to_frame()
loans_skippers_df.columns = ['skippers']
loans_skippers_df.index.name = 'customer_id'
loans_skippers_df['skippers'] = loans_skippers_df['skippers'].map(count_points_for_skippers)
loans_skippers_df

,skippers
customer_id,
0254558577,0
1737533871,0
1777845416,0


In [299]:
def count_points_for_late_payment(row):
    due_date_to_compare = datetime.strptime(row['due_date'], "%d-%m-%Y")
    payment_date_to_compare = datetime.strptime(row['payment_date'], "%d-%m-%Y")
    if payment_date_to_compare <= due_date_to_compare:
        row['late_payment_point'] = 0
    else:
        row['late_payment_point'] = 1
    return row

In [300]:
loans_df_mark_late_payment = loans_df.apply(count_points_for_late_payment, axis="columns")
loans_df_mark_late_payment.head(2)

,customer_id,first_name,last_name,loan_id,loan_category,nr_of_months,due_amount_in_usd,due_date,payment_date,payment_amount_in_usd,late_payment_point
0,169107568X,Christopher,Turner,08196e933365451cb5f51011d20cc903,Auto,30,258,26-04-2015,2-04-2015,251,0
1,169107568X,Christopher,Turner,08196e933365451cb5f51011d20cc903,Auto,30,258,26-05-2015,12-05-2015,282,0


In [301]:
filt_late_payments = loans_df_mark_late_payment['late_payment_point']!= 0
late_payments_filtered = loans_df_mark_late_payment.loc[filt_late_payments]
late_payments_count = late_payments_filtered['customer_id'].value_counts()
late_payments_df = late_payments_count.to_frame()
late_payments_df.columns = ['late_payments']
late_payments_df.index.name = 'customer_id'
late_payments_df

,late_payments
customer_id,
0254558577,11
1978016417,9
0578871866,8
1259477169,6
0627892167,4
0448103850,3
1767857853,2
1777845416,2


In [302]:
filt_not_full_paid_loans = loans_df['due_amount_in_usd'] > loans_df['payment_amount_in_usd']
not_full_paid_loans_filtered = loans_df.loc[filt_not_full_paid_loans].copy()

In [303]:
def get_date_without_day(date):
    parsed_date = datetime.strptime(date, "%d-%m-%Y")
    return parsed_date.strftime("%m-%Y")

In [304]:
not_full_paid_loans_filtered['m/Y'] = not_full_paid_loans_filtered['due_date'].map(get_date_without_day)

In [305]:
not_full_paid_loans_filtered.head(2)

,customer_id,first_name,last_name,loan_id,loan_category,nr_of_months,due_amount_in_usd,due_date,payment_date,payment_amount_in_usd,m/Y
0,169107568X,Christopher,Turner,08196e933365451cb5f51011d20cc903,Auto,30,258,26-04-2015,2-04-2015,251,04-2015
3,169107568X,Christopher,Turner,08196e933365451cb5f51011d20cc903,Auto,30,258,26-07-2015,7-07-2015,244,07-2015


In [306]:
spend_df['m/Y'] = spend_df['month'].map(get_date_without_day)

In [307]:
spend_df.head(2)

,customer_id,first_name,last_name,spend_rate,available_money_rate,spend_money_in_usd,available_money_in_usd,month,m/Y
0,0254558577,Lori,Guzman,Medium,Medium,1414,540,01-01-2012,01-2012
1,0254558577,Lori,Guzman,Medium,Medium,1650,1464,01-02-2012,02-2012


In [308]:
merged_loans_and_spend = pd.merge(not_full_paid_loans_filtered,spend_df,left_on=['customer_id', 'm/Y'],right_on=['customer_id', 'm/Y'], how='left')
merged_loans_and_spend.tail(2)

,customer_id,first_name_x,last_name_x,loan_id,loan_category,nr_of_months,due_amount_in_usd,due_date,payment_date,payment_amount_in_usd,m/Y,first_name_y,last_name_y,spend_rate,available_money_rate,spend_money_in_usd,available_money_in_usd,month
157,1737533871,Jennifer,Thompson,a222d6646b064dcda27e900a623137ed,Personal,43,188,23-05-2020,2-05-2020,179,05-2020,Jennifer,Thompson,High,Low,3359,363,01-05-2020
158,1737533871,Jennifer,Thompson,a222d6646b064dcda27e900a623137ed,Personal,43,188,23-07-2020,1-07-2020,176,07-2020,Jennifer,Thompson,High,Low,7179,157,01-07-2020


In [309]:
filt_lack_of_full_amount_payment_on_account_available_money = merged_loans_and_spend['payment_amount_in_usd'] < merged_loans_and_spend['available_money_in_usd']
not_full_amount_payment_on_account_available_money = merged_loans_and_spend.loc[filt_lack_of_full_amount_payment_on_account_available_money]

In [310]:
not_full_amount_payment_on_account_available_money_count = not_full_amount_payment_on_account_available_money['customer_id'].value_counts()
not_full_amount_payment_on_account_available_money_df = not_full_amount_payment_on_account_available_money_count.to_frame()
not_full_amount_payment_on_account_available_money_df.columns = ['not_full_paid']
not_full_amount_payment_on_account_available_money_df.index.name = 'customer_id'
not_full_amount_payment_on_account_available_money_df['not_full_paid'] = not_full_amount_payment_on_account_available_money_df['not_full_paid'].map(lambda x: x//3)
not_full_amount_payment_on_account_available_money_df

,not_full_paid
customer_id,
1737533871,8
1259477169,7
0254558577,7
0578871866,6
0627892167,3
1978016417,3
1777845416,3
1767857853,3
169107568X,2


In [311]:
dfs = [loan_debtors_balance_df, loans_skippers_df, late_payments_df, not_full_amount_payment_on_account_available_money_df]
final_df = reduce(lambda left,right: pd.merge(left,right,on=['customer_id'], how='outer'), dfs)

In [312]:
final_df.fillna(0, inplace=True)
final_df['sum_points'] = final_df.sum(axis=1)

In [313]:
final_df.sort_values(by='sum_points', ascending=False)

,debtors,skippers,late_payments,not_full_paid,sum_points
customer_id,,,,,
0254558577,0,0.0,11.0,7,18.0
1978016417,3,0.0,9.0,3,15.0
0578871866,0,0.0,8.0,6,14.0
1259477169,0,0.0,6.0,7,13.0
0448103850,3,0.0,3.0,2,8.0
1737533871,0,0.0,0.0,8,8.0
0627892167,0,0.0,4.0,3,7.0
169107568X,3,0.0,0.0,2,5.0
1767857853,0,0.0,2.0,3,5.0
